In [286]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
%matplotlib inline

In [287]:
names = ['API','Surf_X','Surf_Y','Date_Drilling','Date_Completion','Date_Production','Lateral_Length','Depth_TVD_PPLS','Erosion_PPLS','Pressure_PPLS','TOC_PPLS','Vcarb_PPLS','Vsand_PPLS','Vclay_PPLS','PR_PPLS','YM_PPLS','RHOB_PPLS','Res_PPLS','GR_PPLS','DT_PPLS','DTs_PPLS','Temperature','Temp_Anomaly','S3Tect_PPLS','S3_contrast_PPLS','Heat_Flow','Zone','Nbr_Stages','Frac_Gradient','Proppant_Designed','Proppant_in_Formation','Avg_Breakdown_Pressure','Avg_Treating_Pressure','Max_Treating_pressure','Min_Treating_Pressure','Avg_Rate_Slurry','Max_Rate_Slurry','Min_Rate_Slurry','ShutInPressure_Fil','ShutInPressure_Initial','ISIP','Shot_Density','Shot_Total','Proppant_per_ft','Stage_Spacing','GasCum360','OilCum360']

df_data = pd.read_csv('./TrainSample.csv', 
                        header = None, 
                        sep = ';',
                        decimal = ',',
                        names = names,
                        skiprows = 1,
                        na_filter = True,
                        parse_dates = ['Date_Drilling','Date_Completion','Date_Production'],
                        dtype = {col: np.float32 for col in names}
                       )

df_test = pd.read_csv('./TestSample.csv',
                      header = None, 
                      sep = ';', 
                      decimal = ',', 
                      names = names,
                      skiprows = 1,
                      na_filter = True,
                      parse_dates = ['Date_Drilling','Date_Completion','Date_Production'],
                      dtype = {col: np.float32 for col in names}
                     )

#df_data.set_index('API')

df_test.head(5)

,API,Surf_X,Surf_Y,Date_Drilling,Date_Completion,Date_Production,Lateral_Length,Depth_TVD_PPLS,Erosion_PPLS,Pressure_PPLS,...,Min_Rate_Slurry,ShutInPressure_Fil,ShutInPressure_Initial,ISIP,Shot_Density,Shot_Total,Proppant_per_ft,Stage_Spacing,GasCum360,OilCum360
0,633.0,0.201275,-0.577329,2013-06-23,2013-10-26,2014-10-01,-0.694826,0.641631,-0.356227,1.410748,...,0.667402,NaN,-0.740473,0.131374,0.187082,2.409201,0.160140,-0.710739,NaN,NaN
1,587.0,0.222609,0.670307,2012-02-06,2012-03-07,2013-05-08,-0.479516,-1.377997,-1.812483,1.605136,...,0.285738,-0.919600,-1.103957,-1.282464,0.667339,-0.560750,-1.149666,2.085649,NaN,NaN
2,264.0,0.022105,0.404387,2013-05-19,2013-07-24,2013-06-10,0.310782,-0.775745,0.243462,-0.136014,...,0.459046,0.614126,1.029028,1.103339,-0.773433,0.924226,-0.585869,-0.137427,NaN,NaN
3,680.0,0.384359,0.372793,2012-08-18,2012-12-19,2013-12-06,-0.624508,-1.082974,-1.922374,1.443630,...,0.332100,-0.782232,0.450368,-0.136076,0.667339,-0.560750,0.595622,1.887260,NaN,NaN
4,443.0,-0.138513,0.408803,2013-11-09,2014-03-29,2014-11-05,0.338785,-0.616106,0.556826,-1.011425,...,0.323490,NaN,0.743256,0.835687,-1.253690,0.181738,0.151633,-0.216512,NaN,NaN


In [288]:
##Définition des labels

y = {'GasCum360' : df_data['GasCum360'],
     'OilCum360' : df_data['OilCum360']}

df_target = pd.DataFrame(y)

In [289]:
cols_date = ['Date_Drilling','Date_Completion','Date_Production', 'GasCum360', 'OilCum360']

df_data.drop(cols_date, 1, inplace=True)
df_test.drop(cols_date, 1, inplace=True)

In [290]:
cols_with_nan = ['Pressure_PPLS','Nbr_Stages','Frac_Gradient','Proppant_Designed','Proppant_in_Formation','Avg_Breakdown_Pressure','Avg_Treating_Pressure','Max_Treating_pressure','Min_Treating_Pressure','Avg_Rate_Slurry','Max_Rate_Slurry','Min_Rate_Slurry','ShutInPressure_Fil','ShutInPressure_Initial','ISIP','Shot_Density','Shot_Total','Proppant_per_ft','Stage_Spacing']

df_data.drop(cols_with_nan, axis=1, inplace=True)
df_test.drop(cols_with_nan, axis=1, inplace=True)

In [291]:
from sklearn import cross_validation

X_train, X_test, y_train, y_test = cross_validation.train_test_split(df_data, df_target, test_size=0.2, random_state=0)

In [292]:
#x_testFixed = x_test.fillna(x_test.median())
#x_trainFixed = x_train.fillna(x_train.median())

In [293]:
print(X_train.shape, y_train.shape)

(368, 23) (368, 2)


In [294]:
## Fit regression model

regr_1 = KNeighborsRegressor()
regr_1.fit(X_train, y_train)

## Predict

y_predictDec = regr_1.predict(X_test)

In [295]:
## Metrics training

from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_predictDec) 
#mean_squared_error(y_test, y_predictAda)



0.94247615

In [296]:
gascumpred = []
oilcumpred = []

gascumpred = [element[0] for element in y_predictFin]
oilcumpred = [element[1] for element in y_predictFin]

In [297]:
gascumpred_max = max(gascumpred)
gascumpred_min = min(gascumpred)

oilcumpred_max = max(oilcumpred)
oilcumpred_min = min(oilcumpred)

In [298]:
GasCum360_inf = gascumpred - (gascumpred_max - gascumpred_min) / 4
GasCum360_sup = gascumpred + (gascumpred_max - gascumpred_min) / 4

OilCum360_inf = oilcumpred - (oilcumpred_max - oilcumpred_min) / 4
OilCum360_sup = oilcumpred + (oilcumpred_max - oilcumpred_min) / 4


In [299]:
print(GasCum360_inf[:10], GasCum360_sup[:10])

[-1.69242465 -0.7362999  -1.28992261 -0.78355551 -1.04437913 -2.07812074
 -1.54112706 -1.91718182  0.59632191  0.59632191] [ 0.79845613  1.75458089  1.20095817  1.70732528  1.44650166  0.41276005
  0.94975373  0.57369897  3.0872027   3.0872027 ]


In [300]:
-57 < -36

True

In [301]:
GasCum360_inf.tolist
GasCum360_sup.tolist

OilCum360_inf.tolist
OilCum360_sup.tolist

<function ndarray.tolist>

In [302]:
## Output

"""
A FAIRE

Faire une cellule avec le classifier qui prend en entrée toutes les données
du dataset d'entrainement.
Faire la prédiction sur le dataset de test
Mettre en forme les données ainsi prédite pour les exporter dans un CSV

A FAIRE 
"""

id_test = df_test['API'].values.tolist()

output = pd.DataFrame({'API': id_test,
                       'GasCum360_INF': GasCum360_inf,
                       'GasCum360_SUP': GasCum360_sup,
                       'OilCum360_INF': OilCum360_inf,
                       'OilCum360_SUP': OilCum360_sup},
                      index=id_test
                     )

output.head()

output.to_csv('coche-julien-challenge-total.csv', index=False, sep= ';', decimal=',')

In [303]:
"""
Rappport 1 :

Importation correcte des données
Début du travail exploratoire, affichage du head et description des données
Beaucoup de colonnes sont incomplètes => identification et élimination de ces colonnes
Observation des composantes corrélées.

Rapport 2 :

Elimination des colonnes incomplètes
Exploration des données de sorties
Choix de l'algorithme - DecisienTreeRegressor / AdaBoost

Rapport 3 :

Fin elimination des colonnes incompletes
Fin du formatage des données
Debut mise en place de la cross_validation
Debut mise en place des metrics
"""

"\nRappport 1 :\n\nImportation correcte des données\nDébut du travail exploratoire, affichage du head et description des données\nBeaucoup de colonnes sont incomplètes => identification et élimination de ces colonnes\nObservation des composantes corrélées.\n\nRapport 2 :\n\nElimination des colonnes incomplètes\nExploration des données de sorties\nChoix de l'algorithme - DecisienTreeRegressor / AdaBoost\n\nRapport 3 :\n\nFin elimination des colonnes incompletes\nFin du formatage des données\nDebut mise en place de la cross_validation\nDebut mise en place des metrics\n"

In [304]:
output_test.describe()

,API,GasCum360_inf,GasCum360_sup,OilCum360_inf,OilCum360_sup
count,235.000000,235.000000,235.000000,235.000000,235.000000
mean,392.697876,2.929953,1.734154,2.461004,1.614191
std,231.284973,0.994614,0.994614,0.977906,0.977906
min,3.000000,1.725680,0.529881,1.636576,0.789764
25%,183.500000,2.267317,1.071518,1.699767,0.852955
50%,394.000000,2.565136,1.369337,2.124407,1.277595
75%,590.500000,3.469317,2.273518,2.904495,2.057683
max,786.000000,7.568637,6.372838,6.603354,5.756541
